In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
from pandas import DataFrame
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import History 
history = History()

In [ ]:
# input the name of dataset
# Example: Kefiran
name_input = input('input name :')

In [ ]:
# import the nomalized of the dataset
address = 'norm_file/norm_%s.csv' %(name_input)
dataset = pd.read_csv(address)

In [ ]:
# X is the features of the ANN model
# Y is the target of the ANN model
X = dataset.drop(['OD'], axis = 1)
y = dataset['OD']

In [ ]:
# Split the dataset to training and testing data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

# Save the training and tesing dataset in the test_train folder
# Must create the folder before start running the program
X_train.to_csv('test_train/%s_X_train.csv' %(name_input))
X_test.to_csv('test_train/%s_X_test.csv' %(name_input))
y_train.to_csv('test_train/%s_y_train.csv' %(name_input))
y_test.to_csv('test_train/%s_y_test.csv' %(name_input))

In [ ]:
# The function for difind the number of neurons per each hiddinlayer
# And defind the activation function
def num_neuron(n1, n2, n3, act):
    model.add(Dense(n1,input_shape=(4,), activation=act))
    model.add(Dense(n2,activation=act))
    model.add(Dense(n3,activation=act))
    model.add(Dense(1,activation='linear'))

# The function for calculate the error value of the model
# Including MSE, MAE, and PMAE
def cal_error(ypred, ytest):
    mse = 0
    mae = 0
    percenterror = 0
    for i in range(0, len((ypred))):
        mse += (ypred[i] - ytest[i])**2
        mae += abs(ypred[i] - ytest[i])
        percenterror += (abs(ypred[i] - ytest[i]))*100/ytest[i]
    mse = mse/len(ypred)
    mae = mae/len(ypred)
    rmse = np.sqrt(mse)
    percenterror = percenterror/len(ypred)
    error_list = [mae, mse, rmse, percenterror]
    return error_list

In [ ]:
# This is the brute force programming to create the ANN model with 3 hidden layer.
# By each layer contain the neurons between 4 and 30 per each layer.
# With the activation function including relu, sigmoid, and tanh.

nlable = []        # The list for collect the name of model.
error_list = []   # The list for collect the error value of the model.
func = ['relu', 'sigmoid', 'tanh'] #List of the activation fucntion wich used to train the model.
for act in func:
    for i in range(4, 31):
        for j in range(4, 31):
            for k in range(4, 31):
                
                model = Sequential()
                num_neuron(i, j, k, act) # Defind the structure of the model
                model.compile(loss='mse', optimizer='adam') #start the training process
                hist = model.fit(X_train,y_train,validation_data=(X_test, y_test),epochs=500) 
                train_history = DataFrame(hist.history) # Collect the loss of training and validation dataset per epoch
                
                # Predict the testing dataset by using the trained model
                y_pred = model.predict(X_test) 
                
                # change y_pred and y_test to list because it easier to change it to dataframe in the future 
                y_testlistt=y_test.values.tolist()
                y_predlistt = []
                for l in range(0,len(y_pred)):
                    y_predlistt.append(y_pred[l][0])
            
                # Calculate the error between the prediction value and the testing value and collect it in the list
                error_list.append(cal_error(y_predlistt, y_testlistt))
                
                # named the model by the number of neuron and activation function
                # And collect name of the model for label the error value of this model in dataframe
                model_name = '%s_%d_%d_%d' %(act, i, j, k) 
                nlable.append(model_name)
                
                # The model save in the folder wich named by the input material
                address_save = 'model_%s/%s_%s.h5' %(name_input, name_input, model_name)
                model.save(address_save)
                
                # Save training and validation loss
                # The Folder of the loss is in the model folder by the name of folder is "loss"
                address_loss = 'model_%s/loss/%s_loss_%s.csv' %(name_input, name_input, model_name)
                train_history.to_csv(address_loss)
                
                # Save error_data to the folder "error_table"
                error_table = DataFrame(error_list,columns=['mae', 'mse', 'rmse', 'P_error'])
                error_table['neuron'] = nlable
                error_table = error_table[['neuron', 'mae', 'mse', 'rmse', 'P_error']]
                error_tablee = error_table.sort_values(by=['P_error'])
                address = 'error_table/%s_error.csv' %(name_input)
                error_tablee.to_csv(address)